In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ta

In [15]:
dataset_train = pd.read_csv('train.csv')
dataset_train = dataset_train.drop(["Date","Turnover (Lacs)"], axis=1)
RSI=ta.momentum.rsi(close=dataset_train['Close'])
MFI=ta.volume.money_flow_index(dataset_train['High'], dataset_train['Low'], dataset_train['Close'], dataset_train['Total Trade Quantity'],n=14)
dataset_train["RSI"]= RSI
dataset_train["MFI"]=MFI
dataset_train1= ta.utils.dropna(dataset_train)
dataset_train.head()

,Open,High,Low,Last,Close,Total Trade Quantity,RSI,MFI
0,88.00,88.00,70.00,75.55,75.55,232535736,NaN,NaN
1,76.80,78.60,75.00,75.15,75.20,68865277,NaN,NaN
2,75.10,76.10,74.55,75.25,75.30,37460886,NaN,NaN
3,75.50,75.95,75.00,75.05,75.05,17413433,NaN,NaN
4,75.35,75.40,74.30,74.60,74.45,15961426,NaN,NaN


In [16]:
training_set = dataset_train.iloc[:,:].values
#print(training_set)

In [17]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [18]:
X_train = []
y_train = []
for i in range(60, 2000):
    X_train.append(training_set_scaled[i-60:i])
    y_train.append(training_set_scaled[i, 4])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 8))

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [20]:
regressor = Sequential()

regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], 8)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 1, batch_size = 32)

Epoch 1/1
1940/1940 [==============================] - 6s 3ms/step - loss: nan


In [21]:
dataset_test = pd.read_csv('test.csv')
dataset_test = dataset_test.drop(["Date","Turnover (Lacs)"], axis=1)
RSI=ta.momentum.rsi(close=dataset_test['Close'])
MFI=ta.volume.money_flow_index(dataset_test['High'], dataset_test['Low'], dataset_test['Close'], dataset_test['Total Trade Quantity'],n=14)
dataset_test["RSI"]= RSI
dataset_test["MFI"]=MFI
dataset_test1= ta.utils.dropna(dataset_test)
real_stock_price = dataset_test.iloc[:, 5:6].values

In [28]:
dataset_total = pd.concat((dataset_train1['Close'], dataset_test1['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,8)
inputs = sc.transform(inputs)

In [30]:
X_test = []
for i in range(60, 1000) :
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 8))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

IndexError: tuple index out of range

In [ ]:
plt.plot(real_stock_price, color = 'black', label = 'NTPC Price')
plt.plot(predicted_stock_price, color = 'red', label = 'Predicted NTPC Price')
plt.title('NTPC  Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('NTPC Stock Price')
plt.legend()
plt.show()